# Script that performs scraping from Amazon website of a specific product

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

In [3]:
# Set display options
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', None)  # Prevent wrapping of DataFrame

In [4]:
review_title = []
review_body = []
review_stars = []
i = 0

## Scraping

In [5]:
URL = "https://www.amazon.it/echo-dot-2022/product-reviews/B09B8X9RGM/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

headers = {
        'authority': 'www.amazon.it',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'it-IT,it;en-GB,en-US;q=0.9,en;q=0.8',
    }

while i < 50:
    time.sleep(3)
    webpage = requests.get(URL, headers=headers)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    review_title.append(soup.select('a.review-title'))
    review_body.append(soup.select('div.a-row.review-data span.review-text'))
    review_stars.append(soup.select('div.a-row:nth-of-type(2) > a.a-link-normal:nth-of-type(1)'))
    try:
        print(f'Scraping page {i}')
        next_url = soup.select_one('li.a-last a').get('href')
        URL = f"https://www.amazon.it{next_url}"
        i += 1
    except Exception as e:
        print(f'An error occured {e}')
        

Scraping page 0
Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
An error occured 'NoneType' object has no attribute 'get'
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49


## Text processing

In [7]:
review_title = [[element.text.replace('\n', '') for element in sublist] for sublist in review_title]
review_body = [[element.text.replace('\n', '') for element in sublist] for sublist in review_body]
review_stars = [[element.get('title').split()[0] for element in sublist] for sublist in review_stars] # getting only the number of stars the user put

In [8]:
review_title = [[re.sub("[^a-zA-ZÀ-ÖØ-öø-ÿ]", " ", element) for element in sublist] for sublist in review_title]
review_title = [[element.lower() for element in sublist] for sublist in review_title]

In [9]:
review_body = [[re.sub("[^a-zA-ZÀ-ÖØ-öø-ÿ]", " ", element) for element in sublist] for sublist in review_body]
review_body = [[element.lower() for element in sublist] for sublist in review_body]

In [10]:
df = pd.DataFrame(columns = ['Title', 'Body', 'Stars'])

In [11]:
df['Title'] = [item for sublist in review_title for item in sublist]
df['Body'] = [item for sublist in review_body for item in sublist]
df['Stars'] = [item for sublist in review_stars for item in sublist]

In [12]:
df['Stars'] = [element.replace(',0', '') for element in df['Stars']]
df['Stars'] = df['Stars'].astype(int)
df['Title'] = df['Title'].astype(str)
df['Body'] = df['Body'].astype(str)

## NA's check and file writing

In [13]:
df.isnull().sum()

Title    0
Body     0
Stars    0
dtype: int64

In [14]:
df.sample(10)

,Title,Body,Stars
282,bello,alexa è stratosferica risponde bene ad ogni comando molto comodo e utile per chi sta sempre a letto e per chi perde il telecomando,5
455,alexa,e piccola ma ha un suono spettacolo mi piace un sacco regalate a natale anche hai miei genitori e miei suoceriottimo tuttoimpacchettamwnto spedizione prodotto e consegna ora voglio le lampadine appena potrò me le regalo,5
311,soddisfatto,articolo consegnato nei tempi e rispondente alla descrizione,4
12,utilissimo,per qualità prezzo non si poteva chiedere di meglio,5
425,ottimo prodotto,tutto perfetto ma per installarla ho dovuto chiamare un tecnico non è poi così semplice come dicono,5
238,alexa tutta la vita,ottimo acquisto,5
86,amazon,ottima come tutti prodotti amazon secondo me é da migliore il touch per mettere pausa perché non prende al primo colpo,5
156,miglioramenti solamente nel design,rispetto al modello precedente di eco dot quello di terza generazione personalmente noto solo miglioramenti nel design la qualità audio mi sembra la stessa e l efficienza energetica è pressoché la stessa mi sono trovato bene in passato mi trovo bene tutt ora nulla da aggiungere,5
347,bene,audio discreto a patto di non alzare molto il volume esteticamente accattivante ha un sensore di temperatura che chiedendo informa sui gradi della stanza in cui e collocato senza abbonamento premium si ascoltano i brani casuali e non quelli voluti,5
401,dovrebbero averlo tutti,ne avevo già una in cada e ne ho comprata un altra per mio cognato è un oggetto comodissimo soprattutto se associato ad altri come tv e lampadine non uso più gli interruttori e lo trovo comodissimo è comodo anche per ascoltare musica e come timer può essere usato come vivavoce e per fare chiamate se collegato al telefono eccellente,5


In [15]:
# writing the dataframe into a CSV file just to do not have to scape again in case I do something wrong
df.to_csv('data.csv', index=False)